# Testing Whisper
In this notebook we evaluate Whisper. The notebook assumes that test data is uploaded to an S3 bucket and a endpoint has been created.

In [ ]:
# !pip install evaluate

In [ ]:
# !pip install jiwer

In [1]:
import sagemaker
import boto3

sess = sagemaker.Session()
sagemaker_session_bucket = sess.default_bucket()

In [30]:
s3 = boto3.client('s3')
file_list = []

response = s3.list_objects_v2(
    Bucket=sagemaker_session_bucket,
    Prefix='whisper/data/test/he/')

for content in response.get('Contents', []):
    if "wav" in content['Key']:
        file_list.append(f"s3://{sagemaker_session_bucket}/{content['Key']}")

In [43]:
s3.download_file(sagemaker_session_bucket, "whisper/data/test/he/test-he.tsv", 'test-he.tsv')

In [52]:
import pandas as pd
df = pd.read_csv('test-he.tsv', sep='\t')
df["transcription_whisper"] = ""

In [53]:
df.head()

,id,raw_transcription,transcription,new_filename,transcription_whisper
0,1741,אזורים גדולים צפונית משם הם בעלי אוכלוסייה דלי...,אזורים גדולים צפונית משם הם בעלי אוכלוסייה דלי...,test-he-000.wav,
1,1861,"כתוצאה מכך, השחקנים מעשנים ג'וינטים של קנאביס ...",כתוצאה מכך השחקנים מעשנים ג'וינטים של קנאביס ע...,test-he-001.wav,
2,1803,"הלחות על הידיים שלך תגיב עם השכבות החיצוניות, ...",הלחות על הידיים שלך תגיב עם השכבות החיצוניות מ...,test-he-002.wav,
3,1873,"היו תקיפות בדחיית גברים, ואל תפחדו לעמוד על של...",היו תקיפות בדחיית גברים ואל תפחדו לעמוד על שלכ...,test-he-003.wav,
4,1759,"ד""ר מלאר בלסוברמניאן, בן 29, נמצא בבלו אש, אוה...","ד""ר מלאר בלסוברמניאן בן 29 נמצא בבלו אש אוהיו ...",test-he-004.wav,


In [47]:
endpoint_name = "whisper-large-custom-2022-12-08-21-23-42-385"

In [11]:
from sagemaker.huggingface import HuggingFacePredictor

predictor = HuggingFacePredictor(endpoint_name)

In [74]:
transcription_list = []

for i, file in enumerate(file_list):
    print(i)
    
    data = {
        "s3_file": file,
        "language": "he"
    }
    res = predictor.predict(data=data)
    transcription = res['transcription']
    
    df.at[i, "transcription_whisper"] = transcription

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


In [75]:
df.head()

,id,raw_transcription,transcription,new_filename,transcription_whisper
0,1741,אזורים גדולים צפונית משם הם בעלי אוכלוסייה דלי...,אזורים גדולים צפונית משם הם בעלי אוכלוסייה דלי...,test-he-000.wav,אזורים גדולים צפונית משם הם בעלי אוכלוסייה די...
1,1861,"כתוצאה מכך, השחקנים מעשנים ג'וינטים של קנאביס ...",כתוצאה מכך השחקנים מעשנים ג'וינטים של קנאביס ע...,test-he-001.wav,כתוצאה מכך השחקנים מאשנים ג'וינטים של קנאביס ...
2,1803,"הלחות על הידיים שלך תגיב עם השכבות החיצוניות, ...",הלחות על הידיים שלך תגיב עם השכבות החיצוניות מ...,test-he-002.wav,"הלחות על הידיים שלך תגיב עם משחבות החיצוניות,..."
3,1873,"היו תקיפות בדחיית גברים, ואל תפחדו לעמוד על של...",היו תקיפות בדחיית גברים ואל תפחדו לעמוד על שלכ...,test-he-003.wav,לא תקיפו על ביטחיית גברים ואל תפחדו ללמוד לדש...
4,1759,"ד""ר מלאר בלסוברמניאן, בן 29, נמצא בבלו אש, אוה...","ד""ר מלאר בלסוברמניאן בן 29 נמצא בבלו אש אוהיו ...",test-he-004.wav,"ד.מ.ב.מ.ע.ב.29 נמצא בבלו אש, הוהיו. פרוור נמצ..."


In [76]:
import re

def clean(text):
    return re.sub(r'[^\w\s]', '', text)

In [77]:
predictions = list(df["transcription_whisper"])
references = list(df["raw_transcription"])

In [78]:
predictions_clean = list(map(clean, predictions))
references_clean = list(map(clean, references))

In [83]:
import evaluate
wer = evaluate.load("wer")

In [84]:
wer_score = wer.compute(predictions=predictions, references=references)
wer_score

0.3350485991995426

In [85]:
wer_score = wer.compute(predictions=predictions_clean, references=references_clean)
wer_score

0.28219805380652546